<a href="https://colab.research.google.com/github/nspiegeln/iml_2022/blob/main/task4_main2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import pandas as pd
import torch
import numpy as np
from torch import nn
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [76]:
#load data
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [89]:
path = "gdrive/MyDrive/task4/"
df_train= pd.read_csv(path + "train_features.csv")
df_train_labels= pd.read_csv(path + "train_labels.csv")
df_pretrain= pd.read_csv(path +"pretrain_features.csv")
df_pretrain_labels= pd.read_csv(path +"pretrain_labels.csv")
df_test= pd.read_csv(path +"test_features.csv")

In [90]:
df_train_values = df_train.iloc[:,2::]
df_pretrain_values = df_pretrain.iloc[:,2::]

df_train_lab = df_train_labels.iloc[:,1]
df_pretrain_lab = df_pretrain_labels.iloc[:,1]

train = df_train_values.values #dataset with known homo-lumo gap
pretrain = df_pretrain_values.values #dataset with known lumo value

train_labels = df_train_lab.values
pretrain_labels = df_pretrain_lab.values

In [91]:
x_pretrain, x_pretest, y_pretrain, y_pretest = train_test_split(pretrain, pretrain_labels, test_size = 0.1)
print(x_pretrain.shape[0], x_pretrain.shape[1])
print(x_pretest.shape[0], x_pretest.shape[1])

45000 1000
5000 1000


In [92]:
#get device
def get_default_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()

In [93]:
# # PRE-PROCESSING of features according to autoencoder
# # pre-train set

# #hyperparameters
# dim = 50
# epochs = 30 # best results for epochs= 5 for dim = 200

# # create encoder and decoder to reduce dimensionality of features
# class Autoencoder(Model):
#   def __init__(self, dim):
#     super(Autoencoder, self).__init__()
#     self.dim = dim   
#     self.encoder = tf.keras.Sequential([
#       layers.Flatten(),
#       layers.Dense(dim, activation='relu'),
#     ])
#     self.decoder = tf.keras.Sequential([
#       layers.Dense(1000, activation='sigmoid')
#     ])

#   def call(self, x):
#     encoded = self.encoder(x)
#     decoded = self.decoder(encoded)
#     return decoded

# autoencoder = Autoencoder(dim)

# autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

# autoencoder.fit(x_pretrain, x_pretrain, epochs=epochs, shuffle=True, validation_data=(x_pretest, x_pretest))

In [94]:
# # hyperparameters
# output_dim = 1
# epochs = 5        # doesn't decrease after 5, bad loss
# batch_size = 64

# # predict one LUMO label
# class Predictor(Model):
#     def __init__(self, output_dim):
#       super(Predictor, self).__init__()
#       self.output_dim = output_dim   
#       self.predictor = tf.keras.Sequential([
#         layers.Flatten(),
#         layers.Dense(50, activation='relu'),
#         layers.Dense(25, activation='relu'),
#         layers.Dense(5, activation='relu'),
#         layers.Dense(output_dim, activation='relu'),
#       ])

#     def call(self, x):
#       predicted = self.predictor(x)
#       return predicted
    
# predict = Predictor(output_dim)
# predict.compile(optimizer='adam', loss=losses.MeanSquaredError())
# predict.fit(z_encoded_pretrain, y_pretrain, batch_size = batch_size, epochs=epochs, shuffle=True) #, validation_data=(x_encoded_pretest, y_pretest))

In [96]:
# PRE-PROCESSING of features according to autoencoder
# pre-train set

#hyperparameters
dim = 200
epochs = 100         #resuls still decrease after epoch 100,  @epoch=60 loss=4.7e-3, @epoch=100 loss=3.4e-3
batch_size = 64

# create encoder and decoder to reduce dimensionality of features
class Autoencoder(Model):
  def __init__(self, dim):
    super(Autoencoder, self).__init__()
    self.dim = dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(500, activation='relu'),
      layers.Dense(dim, activation='relu'),
    ])
    self.predictor = tf.keras.Sequential([
      layers.Dense(100, activation='relu'),
      layers.Dense(50, activation='relu'),
      layers.Dense(1),
    ])

  def call(self, x):
    encoded = self.encoder(x)
    predicted = self.predictor(encoded)
    return predicted

autoencoder = Autoencoder(dim)
sgd = SGD(learning_rate=0.1, decay=1e-2, momentum=0.9)
autoencoder.compile(optimizer='sgd', loss=losses.MeanSquaredError())

autoencoder.fit(x_pretrain, y_pretrain, epochs=epochs, batch_size = batch_size, shuffle=True, validation_data=(x_pretest, y_pretest))

Epoch 1/100
704/704 [==============================] - 23s 30ms/step - loss: 0.0866 - val_loss: 0.0279
Epoch 2/100
704/704 [==============================] - 23s 33ms/step - loss: 0.0249 - val_loss: 0.0223
Epoch 3/100
704/704 [==============================] - 18s 25ms/step - loss: 0.0200 - val_loss: 0.0219
Epoch 4/100
704/704 [==============================] - 17s 24ms/step - loss: 0.0173 - val_loss: 0.0171
Epoch 5/100
704/704 [==============================] - 20s 28ms/step - loss: 0.0156 - val_loss: 0.0181
Epoch 6/100
704/704 [==============================] - 15s 22ms/step - loss: 0.0143 - val_loss: 0.0140
Epoch 7/100
704/704 [==============================] - 35s 50ms/step - loss: 0.0133 - val_loss: 0.0132
Epoch 8/100
704/704 [==============================] - 24s 34ms/step - loss: 0.0125 - val_loss: 0.0154
Epoch 9/100
704/704 [==============================] - 15s 21ms/step - loss: 0.0118 - val_loss: 0.0120
Epoch 10/100
704/704 [==============================] - 21s 30ms/step - l

In [ ]:
autoencoder.encoder.summary()
autoencoder.predictor.summary()

In [ ]:
y_predpretrain = autoencoder.predict(x_pretrain)


In [ ]:
# Transfer learning to HOMO-LUMO gap learning
# freeze encoder features and train predictor layer

# run features through encoder
encodedtrain = autoencoder.encoder(train).numpy()
y_predicttrain = autoencoder.predict(train)

In [ ]:


autoencoder.layers[-1].output






In [ ]:

#hyperparameters
dim = 1
epochs = 100         # epoch = 100 loss = 0.181
#batch_size = 64
# create encoder and decoder to reduce dimensionality of features
class Homolumo(Model):
  def __init__(self, dim):
    super(Homolumo, self).__init__()
    self.dim = dim   
    self.inflator = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(50, activation='relu'),
      layers.Dense(70, activation='relu'),
      layers.Dense(80, activation='relu'),
      layers.Dense(100, activation='relu'),
    ])
    self.deflator = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(100, activation='relu'),
      layers.Dense(80, activation='relu'),
      layers.Dense(70, activation='relu'),
      layers.Dense(50, activation='relu'),
      layers.Dense(dim),
    ])

  def call(self, x):
    inflated = self.inflator(x)
    deflated = self.deflator(inflated)
    return deflated

model = Homolumo(dim)

sgd = SGD(learning_rate=0.1, decay=1e-2, momentum=0.9)

model.compile(optimizer='sgd', loss=losses.MeanSquaredError())

model.fit(y_predicttrain, train_labels, epochs=epochs, shuffle=True) #, validation_data=(y_predicttest, y_pretest))

In [ ]:
model.inflator.summary()
model.deflator.summary()

In [81]:
#predict homolumo gap for testset
x_test = df_test.iloc[:,2::]
id_test = df_test.iloc[:,0]

y_predicttest = autoencoder.predict(x_test)
y_test = model.predict(y_predicttest)

In [87]:
#export solution
df_evaltest = pd.DataFrame(y_test, columns = ['y'])
df_evaltest.insert(0, 'Id', id_test)
df_evaltest.to_csv('solutiontot.csv', index=False, index_label=False, encoding = 'utf-8')

In [88]:
print(df_evaltest)

         Id         y
0     50100  1.751833
1     50101  1.763083
2     50102  1.838370
3     50103  1.667388
4     50104  1.844060
...     ...       ...
9995  60095  1.655768
9996  60096  1.870003
9997  60097  1.908120
9998  60098  1.717433
9999  60099  1.862450

[10000 rows x 2 columns]
